In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C://Users//Admin//Desktop//github//ID2223//project//air_qual_data.csv", delimiter=",")

In [3]:
df.columns = df.columns.str.replace(" ", "")
df.drop(columns=['no2', 'co'], inplace=True)
df.replace(" ", "", inplace=True)
df['date'] = pd.to_datetime(df['date'])

In [9]:
df.mask(df== "", inplace=True) # replace empty strings with NaNs

df['pm25'] = pd.to_numeric(df['pm25']) # convert strings to numbers
df['pm10'] = pd.to_numeric(df['pm10'])

pm25_mean = int(df['pm25'].mean(skipna=True)) # column means
pm10_mean = int(df['pm10'].mean(skipna=True))

df['pm25'].fillna(pm25_mean, inplace=True) # missing value imputation
df['pm10'].fillna(pm10_mean, inplace=True)

df['pm10'] = df['pm10'].astype(int) # the measurements should be integers
df['pm25'] = df['pm25'].astype(int)

In [9]:
df['aqi'] = df[['pm25', 'pm10']].max(axis=1) # compute AQI

In [10]:
dates = pd.Series(pd.date_range(start='2013-12-31', end='2022-12-22')) # identify missing days
missing_dates = pd.Series(dates[dates.isin(df['date']) == False])
missing_dates

363    2014-12-29
364    2014-12-30
730    2015-12-31
731    2016-01-01
1345   2017-09-06
1346   2017-09-07
1347   2017-09-08
1348   2017-09-09
1747   2018-10-13
2190   2019-12-30
2281   2020-03-30
2594   2021-02-06
dtype: datetime64[ns]

In [11]:
extra_dates = missing_dates.to_frame(name = "date")
extra_dates['pm25'] = pm25_mean
extra_dates['pm10'] = pm10_mean
extra_dates['aqi'] = max(pm25_mean, pm10_mean)

In [ ]:
df = pd.concat([df, extra_dates])

In [13]:
df.sort_values(by=['date'], ignore_index=True, inplace=True)

In [20]:
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d')) #convert because Hopsworks doesn't accept datetime as key

In [ ]:
import hopsworks
import os

os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE'] = '1'
client= hopsworks.login()
stream = client.get_feature_store()

aqi_dt = stream.get_or_create_feature_group(
    name = 'air_quality_fg',
    description = 'Air quality characteristics of each day',
    version=3,
    primary_key=['date'],
    online_enabled=True
)
aqi_dt.insert(df)

## Historical AQI data EDA summary

- only 5 columns: date, pm10, pm2.5, no2, co
- records go back as far as 2014
- dropped no2 and co columns as we were allowed not to use it
- aqi = max(pm10, pm2.5) as new column
- deleted exta spaces before and after records
- replaced missing records with NaNs
- original data was string, convert numeric values to integers, dates to datetime
- input missing dates with NaNs
- sort the records by date